In [8]:
#Below is an integrated Python script that combines the extraction
#logic from PDFs and the application of a trained Camembert model for NER.
import pdfplumber
import re
import os
from transformers import CamembertTokenizerFast, CamembertForTokenClassification, pipeline

# Define the path to the PDF files
pdf_folder = r"C:\Users\User\Documents\NLP"
pdf_files = [f"CR-urgence-{i}.pdf" for i in range(1, 6)]

model_path = "./saved_model"

# Load the fine-tuned Camembert model for NER
tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")
model = CamembertForTokenClassification.from_pretrained(model_path)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Improve regex patterns for extraction
symptoms_patterns = re.compile(
    r'\b(f(ièvre|ebril)|[ée]tat fébrile|toux|mal de gorge|céphalée|rhume|rhinit|congestion|tremblement|'
    r'(douleur|oppression) thoracique|essoufflement|dyspn[ée]e|tachypn[ée]e)\b', 
    re.IGNORECASE
)
smoking_patterns = re.compile(r'\b(fumeur|non-fumeur|tabac|cigarettes)\b', re.IGNORECASE)
comorbidities_patterns = re.compile(
    r'\b(diabète|hypertension|asthme|BPCO|cancer|maladie cardiaque|'
    r'chronique|insuffisance cardiaque|pathologie respiratoire|obésité|'
    r'sleeve|chirurgie bariatrique|hypersensibilité|allergie|'
    r'HTA|chirurgie|antécédents|effexor|psychiatrique)\b', 
    re.IGNORECASE
)

# Function to apply NER and regex for text extraction
def extract_info(pdf_path):
    extracted_data = {
        "Symptômes": [],
        "Statut Tabagique": [],
        "Comorbidités": [],
        "NER_Entities": []
    }
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                text = text.replace('\n', ' ').strip()  # Remove line breaks for smoother analysis
                text = re.sub(r'\s+', ' ', text)  # Reduce multiple spaces to a single space
                
                # Apply regex patterns
                symptoms = symptoms_patterns.findall(text)
                extracted_data["Symptômes"].extend(symptoms)
                
                smoking_status = smoking_patterns.findall(text)
                extracted_data["Statut Tabagique"].extend(smoking_status)
                
                comorbidities = comorbidities_patterns.findall(text)
                extracted_data["Comorbidités"].extend(comorbidities)
                
                # Apply NER model
                ner_results = ner_pipeline(text)
                for entity in ner_results:
                    extracted_data["NER_Entities"].append({
                        "entity": entity["entity_group"],
                        "word": entity["word"],
                        "start": entity["start"],
                        "end": entity["end"],
                        "score": entity["score"]
                    })
    
    # Deduplicate and format results
    extracted_data["Symptômes"] = list(set(extracted_data["Symptômes"]))
    extracted_data["Statut Tabagique"] = list(set(extracted_data["Statut Tabagique"]))
    extracted_data["Comorbidités"] = list(set(extracted_data["Comorbidités"]))
    extracted_data["NER_Entities"] = list({(e['word'], e['entity']): e for e in extracted_data["NER_Entities"]}.values())
    
    return extracted_data

# Process each PDF and compile the results
all_extracted_data = []
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf_file)
    extracted_info = extract_info(pdf_path)
    all_extracted_data.append(extracted_info)

# Display the extracted data for review
for idx, data in enumerate(all_extracted_data, 1):
    print(f"Données extraites de {pdf_files[idx-1]}:")
    print(f"Symptômes liés à la grippe: {data['Symptômes']}")
    print(f"Statut Tabagique: {data['Statut Tabagique']}")
    print(f"Comorbidités: {data['Comorbidités']}")
    #print(f"Entités NER: {data['NER_Entities']}\n")


Données extraites de CR-urgence-1.pdf:
Symptômes liés à la grippe: [('oppression thoracique', '', 'oppression'), ('tachypnée', '', ''), ('dyspnée', '', ''), ('douleur thoracique', '', 'douleur')]
Statut Tabagique: ['Tabac']
Comorbidités: ['asthme', 'Antécédents', 'insuffisance cardiaque']
Données extraites de CR-urgence-2.pdf:
Symptômes liés à la grippe: []
Statut Tabagique: []
Comorbidités: ['diabète', 'Antécédents']
Données extraites de CR-urgence-3.pdf:
Symptômes liés à la grippe: [('dyspnée', '', ''), ('douleur thoracique', '', 'douleur')]
Statut Tabagique: []
Comorbidités: ['sleeve', 'hypersensibilité', 'Obésité', 'Antécédents', 'HTA', 'allergie', 'effexor', 'Sleeve']
Données extraites de CR-urgence-4.pdf:
Symptômes liés à la grippe: [('douleur thoracique', '', 'douleur')]
Statut Tabagique: []
Comorbidités: ['allergie', 'Antécédents', 'chronique']
Données extraites de CR-urgence-5.pdf:
Symptômes liés à la grippe: [('fièvre', 'ièvre', '')]
Statut Tabagique: ['Tabac']
Comorbidités: 